In [178]:
# Import libraries
import argparse
import time
from collections import Counter
from typing import List
import numpy as np
import math


# Define a class to store a single sentiment example
class SentimentExample:
    def __init__(self, words, label):
        self.words = words
        self.label = label

    def __repr__(self):
        return repr(self.words) + "; label=" + repr(self.label)

    def __str__(self):
        return self.__repr__()


# Reads sentiment examples in the format [0 or 1]<TAB>[raw sentence]; tokenizes and cleans the sentences.
def read_sentiment_examples(infile):
    f = open(infile, encoding="iso8859")
    exs = []
    for line in f:
        fields = line.strip().split(" ")
        label = 0 if "0" in fields[0] else 1
        words = fields[1:]
        exs.append(SentimentExample(words, label))
    f.close()
    return exs


# Bijection between objects and integers starting at 0. Useful for mapping
# labels, features, etc. into coordinates of a vector space.
# This class creates a mapping between objects (here words) and unique indices
# For example: apple->1, banana->2, and so on
class Indexer(object):
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}

    def __repr__(self):
        return str([str(self.get_object(i)) for i in range(0, len(self))])

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.objs_to_ints)

    # Returns the object corresponding to the particular index
    def get_object(self, index):
        if index not in self.ints_to_objs:
            return None
        else:
            return self.ints_to_objs[index]

    def contains(self, object):
        return self.index_of(object) != -1

    # Returns -1 if the object isn't present, index otherwise
    def index_of(self, object):
        if object not in self.objs_to_ints:
            return -1
        else:
            return self.objs_to_ints[object]

    # Adds the object to the index if it isn't present, always returns a nonnegative index
    def add_and_get_index(self, object, add=True):
        if not add:
            return self.index_of(object)
        if object not in self.objs_to_ints:
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]


# Feature extraction base type. Takes an example and returns an indexed list of features.
class FeatureExtractor(object):
    # Extract features. Includes a flag add_to_indexer to control whether the indexer should be expanded.
    # At test time, any unseen features should be discarded, but at train time, we probably want to keep growing it.
    def extract_features(self, ex, add_to_indexer):
        raise Exception("Don't call me, call my subclasses")


# Extracts unigram bag-of-words features from a sentence. It's up to you to decide how you want to handle counts
class UnigramFeatureExtractor(FeatureExtractor):
    def __init__(self, indexer: Indexer):
        self.indexer = indexer

    def extract_features(self, ex, add_to_indexer=False):
        features = Counter()
        for w in ex.words:
            feat_idx = (
                self.indexer.add_and_get_index(w)
                if add_to_indexer
                else self.indexer.index_of(w)
            )
            if feat_idx != -1:
                features[feat_idx] += 1.0

        return features


# Bigram feature extractor analogous to the unigram one.
class BigramFeatureExtractor(FeatureExtractor):
    def __init__(self, indexer: Indexer):
        self.indexer = indexer

    def extract_features(self, ex, add_to_indexer=False):
        features = Counter()
        for i in range(len(ex.words) - 1):
            w = ex.words[i] + "||" + ex.words[i + 1]
            feat_idx = (
                self.indexer.add_and_get_index(w)
                if add_to_indexer
                else self.indexer.index_of(w)
            )
            if feat_idx != -1:
                features[feat_idx] += 1.0

        return features


# Sentiment classifier base type
class SentimentClassifier(object):
    # Makes a prediction for the given
    def predict(self, ex: SentimentExample):
        raise Exception("Don't call me, call my subclasses")


class LogisticRegressionClassifier(SentimentClassifier):
    def __init__(
        self,
        feat_extractor: FeatureExtractor,
        train_examples,
        num_iters=10,
        reg_lambda=0.0,
        learning_rate=0.1,
    ):
        self.weights = None  # We will initialize it later based on feature size
        self.bias = 0.0
        self.train(feat_extractor, train_examples, num_iters, reg_lambda, learning_rate)

    def train(
        self,
        feat_extractor: FeatureExtractor,
        train_examples,
        num_iters=10,
        reg_lambda=0.0,
        learning_rate=0.1,
    ):
        ### BEGIN SOLUTION
        self.feat_extractor = feat_extractor
        for sentiment_ex in train_examples:
            _ = self.feat_extractor.extract_features(sentiment_ex, add_to_indexer=True)

        self.weights = np.zeros(len(feat_extractor.indexer))

        for _ in range(num_iters):
            for sentiment_ex in train_examples:
                features_counter = self.feat_extractor.extract_features(sentiment_ex)
                features = np.zeros(len(self.feat_extractor.indexer))

                for feature, count in features_counter.items():
                    features[feature] = count

                z = np.dot(self.weights, features) + self.bias
                prob = 1.0 / (1.0 + np.exp(-z))
                error = sentiment_ex.label - prob

                self.weights += learning_rate * (
                    error * features - reg_lambda * self.weights
                )
                self.bias += learning_rate * error
        return
        ### END SOLUTION

    def predict(self, ex):
        ### BEGIN SOLUTION
        features_counter = self.feat_extractor.extract_features(ex)

        features = np.zeros(len(self.feat_extractor.indexer))
        for feature, count in features_counter.items():
            features[feature] = count

        z = np.dot(self.weights, features) + self.bias
        prob = 1.0 / (1.0 + np.exp(-z))
        return 1 if prob >= 0.5 else 0
        ### END SOLUTION

In [179]:
# Train a logsitic regression model on the given training examples using the given FeatureExtractor
def train_lr(
    train_exs: List[SentimentExample], feat_extractor: FeatureExtractor, reg_lambda
) -> LogisticRegressionClassifier:
    # Initialize the logistic regression classifier with the provided feature extractor,
    # training examples, and other hyperparameters like regularization lambda.
    lr_classifier = LogisticRegressionClassifier(
        feat_extractor,
        train_exs,
        num_iters=10,
        reg_lambda=reg_lambda,
        learning_rate=0.1,
    )
    return lr_classifier


# Main entry point for your modifications. Trains and returns one of several models depending on the options passed
def train_model(feature_type, model_type, train_exs, reg_lambda=0.0):
    # Initialize feature extractor
    if feature_type == "unigram":
        # Add additional preprocessing code here
        feat_extractor = UnigramFeatureExtractor(Indexer())
    elif feature_type == "bigram":
        # Add additional preprocessing code here
        feat_extractor = BigramFeatureExtractor(Indexer())
    else:
        raise Exception("Pass unigram or bigram")

    if model_type == "LogisticRegression":
        model = train_lr(train_exs, feat_extractor, reg_lambda=reg_lambda)
    else:
        raise Exception("Pass LogisticRegression")
    return model


# Evaluates a given classifier on the given examples
def evaluate(classifier, exs):
    return print_evaluation(
        [ex.label for ex in exs], [classifier.predict(ex) for ex in exs]
    )


# Prints accuracy comparing golds and predictions, each of which is a sequence of 0/1 labels.
def print_evaluation(golds, predictions):
    num_correct = 0
    num_pos_correct = 0
    num_pred = 0
    num_gold = 0
    num_total = 0

    if len(golds) != len(predictions):
        raise Exception(
            "Mismatched gold/pred lengths: %i / %i" % (len(golds), len(predictions))
        )

    for idx in range(0, len(golds)):
        gold = golds[idx]
        prediction = predictions[idx]
        if prediction == gold:
            num_correct += 1
        if prediction == 1:
            num_pred += 1
        if gold == 1:
            num_gold += 1
        if prediction == 1 and gold == 1:
            num_pos_correct += 1
        num_total += 1

    return num_correct * 100.0 / num_total

In [180]:
# Load the data from the files
train_exs = read_sentiment_examples("./data/train.txt")
dev_exs = read_sentiment_examples("./data/dev.txt")
n_pos = 0
n_neg = 0
for ex in train_exs:
    if ex.label == 1:
        n_pos += 1
    else:
        n_neg += 1
print("%d train examples: %d positive, %d negative" % (len(train_exs), n_pos, n_neg))
print("%d dev examples" % len(dev_exs))


# Evaluate on train and dev dataset
def eval_train_dev(model):
    train_acc = evaluate(model, train_exs)
    eval_acc = evaluate(model, dev_exs)
    return [train_acc, eval_acc]

6920 train examples: 3610 positive, 3310 negative
872 dev examples


In [181]:
# Evaluate logistic regression with unigram features
unigram_model = train_model("unigram", "LogisticRegression", train_exs)
eval_train_dev(unigram_model)

In [ ]:
# Evaluate logistic regression with bigram features
bigram_model = train_model("bigram", "LogisticRegression", train_exs)
eval_train_dev(bigram_model)

[99.98554913294798, 73.5091743119266]